# Comparaison entre RNAseq Poly(A) et total

In [1]:
# Numeric analysis
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt


C:\Users\33651\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [1]:
import sys
print(sys.version)

3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]


## Récupération des MANE transcrits

On récupère les informations chrom, start, end pour nos 249 gènes d'intérêt.

In [19]:
interet = pd.read_csv('gene_interet', sep="\t")

In [20]:
interet

,#chrom,txStart,txEnd,name2
0,chr1,155610169,155614967,MSTO1
1,chr1,155610169,155614967,MSTO1
2,chr1,155610169,155614967,MSTO1
3,chr1,155610169,155614967,MSTO1
4,chr1,99850488,99924023,AGL
...,...,...,...,...
2344,chr20_KI270871v1_alt,182,33239,KCNQ2
2345,chr20_KI270871v1_alt,182,33239,KCNQ2
2346,chr20_KI270871v1_alt,182,33239,KCNQ2
2347,chr20_KI270871v1_alt,182,33239,KCNQ2


In [21]:
interet_select = interet.drop_duplicates(subset='name2')

In [22]:
interet_select

,#chrom,txStart,txEnd,name2
0,chr1,155610169,155614967,MSTO1
4,chr1,99850488,99924023,AGL
5,chr1,114673097,114695546,AMPD1
6,chr1,201039511,201112426,CACNA1S
7,chr1,94974404,95072951,ALG14
...,...,...,...,...
2189,chr22,19175580,19178736,SLC25A1
2190,chr22,33162227,33920476,LARGE1
2193,chr22,50578962,50582849,CHKB
2202,chr22,35606763,35617329,MB


In [23]:
interet_bed = interet_select.drop(columns=['name2'])

In [ ]:
interet_bed

On récupère un fichier bed. On a maintenant les positions génomiques nécessaires à la sélection des transcrits MANE et MANE SELECT.

In [ ]:
interet_bed.to_csv('interet_bed.txt', sep='\t', index=False)

On sélectionne les MANE transcripts pour nos gènes d'intérêt.

In [ ]:
mane = pd.read_csv('MANE', sep="\t")

In [ ]:
mane

In [ ]:
mane = mane.rename(columns={'#chrom': 'chrom', "geneName2": "symbol", "ncbiId": "NM"})

In [ ]:
mane

In [ ]:
mane_bed = mane.drop(columns=['name','symbol','NM','maneStat'])

In [ ]:
mane_bed

In [ ]:
mane_bed.to_csv('mane.bed', sep='\t', index=False)

In [ ]:
mane.to_csv('mane_status', sep='\t', index=False)

## Positions des exons

On récupère les positions des exons pour nos gènes d'intérêt

In [ ]:
exons = pd.read_csv('exons', sep="\t")

In [ ]:
exons

On sélectionne que les exons codants "NM"

In [ ]:
filtered_exons = exons[exons['#name'].str.startswith('NM')]

In [ ]:
exons_renamed = filtered_exons.rename(columns={'#name':'NM', "name2": "symbol", "txStart":"chromStart","txEnd":"chromEnd"})

In [ ]:
exons_renamed

On fusionne nos deux dataframes en ne gardant que les lignes du dataframe "exons" ayant le même NM que le dataframe "mane"

In [ ]:
mane_exons = pd.merge(exons_renamed,mane['NM'], on='NM')

In [ ]:
mane_exons

Ici, on a donc un nouveau dataframe contenant uniquement les MANE transcripts avec les informations sur nos exons.

On veut créer une colonne contenant les noms des exons et une contenant leurs positions.

Problème : les nombres contenus dans 'exonStarts' et 'exonEnds' sont consiéré comme une chaîne de caractère

In [ ]:
def convert_to_list(exon_starts):
  exon_starts_list = [int(x) for x in exon_starts.split(',') if x.strip()]
  return exon_starts_list


for index, row in mane_exons.iterrows():
    mane_exons.at[index, 'exonStarts'] = convert_to_list(row['exonStarts'])



In [ ]:
for index, row in mane_exons.iterrows():
    mane_exons.at[index, 'exonEnds'] = convert_to_list(row['exonEnds'])

In [ ]:
mane_exons

Maintenant que nos colonnes exonStarts et exonEnds sont des listes (itérables) on peut faire des paires qui correspondent aux positions de nos exons. On restitue également le nom de chaque exon.

In [ ]:
mane_exons_strand = pd.merge(mane_exons,strand)

In [ ]:
mane_exons_strand

In [ ]:
# Initialise une liste pour stocker les nouvelles lignes
new_rows = []

# Initialise le compteur pour chaque identifiant NM
current_nm = None
current_chrom = None

# Parcourt chaque ligne du DataFrame
for index, row in mane_exons_strand.iterrows():
    if row['NM'] != current_nm or row['chrom'] != current_chrom:
        if row['strand'] == '-':
            count = row['exonCount']
        else:
            count = 1
        current_nm = row['NM']
    # Parcourt chaque paire de termes de 'exonStarts' et 'exonEnds'
    for start, end in zip(row['exonStarts'], row['exonEnds']):

        # Crée un dictionnaire représentant une nouvelle ligne avec les colonnes nécessaires
        new_row = {
            'strand': row['strand'],
            'chrom': row['chrom'],
            'chromStart': row['chromStart'],
            'chromEnd': row['chromEnd'],
            'exonCount': row['exonCount'],
            'symbol': row['symbol'],
            'NM': row['NM'],
            'exonStarts': start,
            'exonEnds': end,
            'exon': count
        }
        count += -1 if row['strand'] == '-' else 1
        # Ajoute la nouvelle ligne à la liste
        new_rows.append(new_row)

# Crée un nouveau DataFrame à partir de la liste des nouvelles lignes
positions_exons_strand = pd.DataFrame(new_rows)

# Affiche le DataFrame résultant
positions_exons_strand

In [ ]:
plus = positions_exons_strand.loc[(positions_exons_strand['strand']=='+')]

In [ ]:
plus

In [ ]:
moins = positions_exons_strand.loc[(positions_exons_strand['strand']=='-')]

In [ ]:
moins.head(50)

On vérifie que le compteur d'exon redémarre bien à 1 lorsque l'on change de gène :

In [ ]:
positions_exons.head(20)

In [ ]:
positions_exons_strand

In [ ]:
positions_exons.to_csv('positions_exons', sep='\t', index=False)

## Récupération des données échantillons

A present, on peut récupérer les pronfondeurs par position pour chaque patient. On dispose de 2 jeux de données, un contenant les valeurs obtenues par séquençage mRNAseq polyA (_m)
et un contenant celles obtenues par RNAseq total (_t).

In [ ]:
m548 = pd.read_csv('./2023-71-72/cov-2023-71/2023A548_m.dedup.cov', sep="\t")

In [ ]:
m555 = pd.read_csv('./2023-71-72/cov-2023-71/2023A555_m.dedup.cov', sep="\t")

In [ ]:
m555.tail(10)

Problème : nos dataFrames n'ont pas de noms de colonnes. On veut également ajouter une colonne "échantillons" afin de mieux les identifier par la suite.

In [ ]:
nouveaux_noms_colonnes = ['chrom', 'position', 'profondeur']

In [ ]:
m548 = pd.read_csv('./2023-71-72/cov-2023-71/2023A548_m.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([m548.columns], columns=nouveaux_noms_colonnes)

m548.columns = nouveaux_noms_colonnes

m548 = pd.concat([anciens_noms_colonnes, m548])

m548['échantillon'] = 'm548'

m548.reset_index(drop=True, inplace=True)

In [ ]:
m549 = pd.read_csv('./2023-71-72/cov-2023-71/2023A549_m.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([m549.columns], columns=nouveaux_noms_colonnes)

m549.columns = nouveaux_noms_colonnes

m549 = pd.concat([anciens_noms_colonnes, m549])

m549['échantillon'] = 'm549'

m549.reset_index(drop=True, inplace=True)

In [ ]:
m550 = pd.read_csv('./2023-71-72/cov-2023-71/2023A550_m.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([m550.columns], columns=nouveaux_noms_colonnes)

m550.columns = nouveaux_noms_colonnes

m550 = pd.concat([anciens_noms_colonnes, m550])

m550['échantillon'] = 'm550'

m550.reset_index(drop=True, inplace=True)

In [ ]:
m551 = pd.read_csv('./2023-71-72/cov-2023-71/2023A551_m.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([m551.columns], columns=nouveaux_noms_colonnes)

m551.columns = nouveaux_noms_colonnes

m551 = pd.concat([anciens_noms_colonnes, m551])

m551['échantillon'] = 'm551'

m551.reset_index(drop=True, inplace=True)

In [ ]:
m552 = pd.read_csv('./2023-71-72/cov-2023-71/2023A552_m.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([m552.columns], columns=nouveaux_noms_colonnes)

m552.columns = nouveaux_noms_colonnes

m552 = pd.concat([anciens_noms_colonnes, m552])

m552['échantillon'] = 'm552'

m552.reset_index(drop=True, inplace=True)

In [ ]:
m553 = pd.read_csv('./2023-71-72/cov-2023-71/2023A553_m.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([m553.columns], columns=nouveaux_noms_colonnes)

m553.columns = nouveaux_noms_colonnes

m553 = pd.concat([anciens_noms_colonnes, m553])

m553['échantillon'] = 'm553'

m553.reset_index(drop=True, inplace=True)

In [ ]:
m554 = pd.read_csv('./2023-71-72/cov-2023-71/2023A554_m.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([m554.columns], columns=nouveaux_noms_colonnes)

m554.columns = nouveaux_noms_colonnes

m554 = pd.concat([anciens_noms_colonnes, m554])

m554['échantillon'] = 'm554'

m554.reset_index(drop=True, inplace=True)

In [ ]:
m555 = pd.read_csv('./2023-71-72/cov-2023-71/2023A555_m.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([m555.columns], columns=nouveaux_noms_colonnes)

m555.columns = nouveaux_noms_colonnes

m555 = pd.concat([anciens_noms_colonnes, m555])

m555['échantillon'] = 'm555'

m555.reset_index(drop=True, inplace=True)

In [ ]:
t548 = pd.read_csv('./2023-71-72/cov-2023-72/2023A548_t.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([t548.columns], columns=nouveaux_noms_colonnes)

t548.columns = nouveaux_noms_colonnes

t548 = pd.concat([anciens_noms_colonnes, t548])

t548['échantillon'] = 't548'

t548.reset_index(drop=True, inplace=True)

In [ ]:
t549 = pd.read_csv('./2023-71-72/cov-2023-72/2023A549_t.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([t549.columns], columns=nouveaux_noms_colonnes)

t549.columns = nouveaux_noms_colonnes

t549 = pd.concat([anciens_noms_colonnes, t549])

t549['échantillon'] = 't549'

t549.reset_index(drop=True, inplace=True)

In [ ]:
t550 = pd.read_csv('./2023-71-72/cov-2023-72/2023A550_t.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([t550.columns], columns=nouveaux_noms_colonnes)

t550.columns = nouveaux_noms_colonnes

t550 = pd.concat([anciens_noms_colonnes, t550])

t550['échantillon'] = 't550'

t550.reset_index(drop=True, inplace=True)

In [ ]:
t551 = pd.read_csv('./2023-71-72/cov-2023-72/2023A551_t.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([t551.columns], columns=nouveaux_noms_colonnes)

t551.columns = nouveaux_noms_colonnes

t551 = pd.concat([anciens_noms_colonnes, t551])

t551['échantillon'] = 't551'

t551.reset_index(drop=True, inplace=True)

In [ ]:
t553 = pd.read_csv('./2023-71-72/cov-2023-72/2023A553_t.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([t553.columns], columns=nouveaux_noms_colonnes)

t553.columns = nouveaux_noms_colonnes

t553 = pd.concat([anciens_noms_colonnes, t553])

t553['échantillon'] = 't553'

t553.reset_index(drop=True, inplace=True)

In [ ]:
t554 = pd.read_csv('./2023-71-72/cov-2023-72/2023A554_t.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([t554.columns], columns=nouveaux_noms_colonnes)

t554.columns = nouveaux_noms_colonnes

t554 = pd.concat([anciens_noms_colonnes, t554])

t554['échantillon'] = 't554'

t554.reset_index(drop=True, inplace=True)

In [ ]:
t555 = pd.read_csv('./2023-71-72/cov-2023-72/2023A555_t.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([t555.columns], columns=nouveaux_noms_colonnes)

t555.columns = nouveaux_noms_colonnes

t555 = pd.concat([anciens_noms_colonnes, t555])

t555['échantillon'] = 't555'

t555.reset_index(drop=True, inplace=True)

In [ ]:
t559 = pd.read_csv('./2023-71-72/cov-2023-72/2023A559_t.dedup.cov', sep="\t")

anciens_noms_colonnes = pd.DataFrame([t559.columns], columns=nouveaux_noms_colonnes)

t559.columns = nouveaux_noms_colonnes

t559 = pd.concat([anciens_noms_colonnes, t559])

t559['échantillon'] = 't559'

t559.reset_index(drop=True, inplace=True)

In [ ]:
m555

## Test analyse sur petits échantillions

On va d'abord sélectionner 3 gènes parmis la liste pour procéder à un test. On veut sélectionner les gènes suivants : CAPN3, MTM1, TTN. 

In [ ]:
genes = mane.loc[(mane['symbol'] == 'CAPN3') | (mane['symbol'] == 'MTM1') | (mane['symbol'] == 'TTN')]

genes 

In [ ]:
genes_exons = pd.merge(positions_exons,genes['NM'], on='NM')

genes_exons

In [ ]:
gene_test = genes_exons.iloc[0:5,:]

In [ ]:
gene_test

Test avec deux échantillons (m555,t555)

In [ ]:
liste_df = [m548,m549,m550,m551,m552,m553,m554,m555,t548,t549,t550,t551,t553,t554,t555,t559]

In [ ]:
liste_test = [m555,t555]

In [ ]:
liste_test2 = [m548,t548,m555]

In [ ]:
def traiter_dataframe(df):
    df['profondeur'] = pd.to_numeric(df['profondeur'], errors='coerce')
    df['position'] = pd.to_numeric(df['position'], errors='coerce')
    return df

In [ ]:
def calculer_profondeur_moyenne(row, df):
    exon_profondeurs = df[(df['chrom'] == row['chrom']) & 
                          (df['position'] >= row['exonStarts']) &
                          (df['position'] <= row['exonEnds'])]
    return exon_profondeurs['profondeur'].mean()

In [ ]:
new_rows = []

for index, row in gene_test.iterrows():
    for df in liste_test:
        df = traiter_dataframe(df)
        new_row = {
            'chrom': row['chrom'],
            'chromStart': row['chromStart'],
            'chromEnd': row['chromEnd'],
            'exonCount': row['exonCount'],
            'symbol': row['symbol'],
            'NM': row['NM'],
            'exonStarts': row['exonStarts'],
            'exonEnds': row['exonEnds'],
            'exon': row['exon'],
            'profondeur_moy': calculer_profondeur_moyenne(row,df),
            'échantillon': df['échantillon'].iloc[0],
            'len_exon': (row['exonEnds']-row['exonStarts'])
        }
        new_rows.append(new_row)

gene_test_prof = pd.DataFrame(new_rows)

In [ ]:
gene_test_prof

In [ ]:
gene_test_prof.to_csv('gene_test_prof', sep='\t', index=False)

## Application à tous les échantillons et tous les gènes

In [ ]:
new_rows = []

for index, row in positions_exons.iterrows():
    for df in liste_df:
        df = traiter_dataframe(df)
        new_row = {
            'chrom': row['chrom'],
            'chromStart': row['chromStart'],
            'chromEnd': row['chromEnd'],
            'exonCount': row['exonCount'],
            'symbol': row['symbol'],
            'NM': row['NM'],
            'exonStarts': row['exonStarts'],
            'exonEnds': row['exonEnds'],
            'exon': row['exon'],
            'profondeur_moy': calculer_profondeur_moyenne(row,df),
            'échantillon': df['échantillon'].iloc[0],
            'len_exon': (row['exonEnds']-row['exonStarts'])
        }
        new_rows.append(new_row)

profondeur_exons = pd.DataFrame(new_rows)

In [ ]:
profondeur_exons

In [ ]:
profondeur_exons.to_csv('prodondeur_exons', sep='\t', index=False)

## Gestion des données manquantes

In [ ]:
genes_vides = profondeur_exons.loc[(profondeur_exons['NM'] == 'NM_018847.4') | (profondeur_exons['NM'] == 'NM_014908.4')| (profondeur_exons['NM'] == 'NM_003055.3')| (profondeur_exons['NM'] == 'NM_001137601.3')| (profondeur_exons['NM'] == 'NM_001101362.3')| (profondeur_exons['NM'] == 'NM_019066.5')]

In [ ]:
genes_vides

In [ ]:
pb_exon = profondeur_exons.loc[(profondeur_exons['exonCount']==1)]

In [ ]:
pb_exon.head(50)

On remarque que tous les gènes qui n'ont qu'un exon n'ont pas de graph

In [ ]:
genes_pb = pb_exon['symbol'].unique()

In [ ]:
genes_pb

In [ ]:
MAGEL2 = profondeur_exons.loc[(profondeur_exons['symbol']== 'MAGEL2')]

MAGEL2

In [ ]:
SLC18A3 = profondeur_exons.loc[(profondeur_exons['symbol']== 'SLC18A3')]

SLC18A3

On note également que le gène MAGEL2 n'a pas de données de profondeur pour le transcrit sur le chr15_ML143371v1_fix et que le gène SLC18A3 n'est couvert que pour 2 échantillons (t559,m554).

In [ ]:
moyenne_profondeur_par_gene = profondeur_exons.groupby('symbol')['profondeur_moy'].mean()

top_5_genes = moyenne_profondeur_par_gene.nlargest(5)

print(f'Les 5 gènes avec les valeurs de profondeur moyenne les plus élevées:{top_5_genes}')

In [ ]:
min_5_genes = moyenne_profondeur_par_gene.nsmallest(5)

print(f'Les 5 gènes avec les valeurs de profondeur moyenne les plus faibles:{min_5_genes}')

## Gestion problème strand -

In [ ]:
NM = mane_exons['NM']

In [ ]:
NM = NM.drop_duplicates()

In [ ]:
NM

In [ ]:
NM.to_csv('NM', sep='\t', index=False)

In [ ]:
strand = pd.read_csv('strand.bed', sep="\t")

In [ ]:
strand

In [ ]:
strand = strand.rename(columns={'#strand': 'strand', "name2": "symbol"})

In [ ]:
strand

In [ ]:
profondeur_exons = pd.read_csv('profondeur_exons', sep="\t")

In [ ]:
profondeur_exons.drop(columns=['exon'])

In [ ]:
merged_df = pd.merge(profondeur_exons, positions_exons_strand, on=['exonStarts', 'exonEnds', 'NM'], how='inner')

merged_df.rename(columns={'exon_y': 'exon'}, inplace=True)

merged_df.drop('exon_x', axis=1, inplace=True)

merged_df

In [ ]:
merged_df.drop(columns=['chrom_y', 'chromStart_y', 'chromEnd_y', 'exonCount_y', 'symbol_y'], inplace=True)

In [ ]:
merged_df.rename(columns={'chrom_x': 'chrom', "chromStart_x":'chromStart', "chromEnd_x":'chromEnd',"exonCount_x": "exonCount",'symbol_x':'symbol'})

In [ ]:
profondeur_exons_strand = merged_df.rename(columns={'chrom_x': 'chrom', "chromStart_x":'chromStart', "chromEnd_x":'chromEnd',"exonCount_x": "exonCount",'symbol_x':'symbol'})

In [ ]:
profondeur_exons_strand

In [ ]:
profondeur_exons_strand.to_csv('profondeur_exons_strand', sep='\t', index=False)

## Application des groupes

In [27]:
groupe = pd.read_csv('groupe', sep="\t")

In [28]:
groupe

,Description,Groupe
0,MSTO1,Groupe 1
1,AGL,Groupe 3
2,AMPD1,Groupe 3
3,CACNA1S,Groupe 3
4,ALG14,Groupe 1
...,...,...
300,SLC25A1,Groupe 2
301,LARGE1,Groupe 2
302,CHKB,Groupe 2
303,MB,Groupe 3


In [29]:
groupe = groupe.rename(columns={"Description": "symbol"})

In [30]:
profondeur_grouped = pd.merge(groupe, profondeur_exons_norm, on=['symbol'], how='right')

In [31]:
profondeur_grouped

,symbol,Groupe,chrom,chromStart,chromEnd,exonCount,NM,exonStarts,exonEnds,profondeur_moy,échantillon,len_exon,strand,exon,prof_total,ratio,profondeur_norm
0,AMPD1,Groupe 3,chr1,114673097,114695546,16,NM_000036.3,114673097,114673272,825.828571,m548,175,-,16,153,1.000000,825.828571
1,AMPD1,Groupe 3,chr1,114673097,114695546,16,NM_000036.3,114673097,114673272,648.268571,m549,175,-,16,169,1.104575,586.894032
2,AMPD1,Groupe 3,chr1,114673097,114695546,16,NM_000036.3,114673097,114673272,631.280000,m550,175,-,16,"159,9",1.045098,604.039025
3,AMPD1,Groupe 3,chr1,114673097,114695546,16,NM_000036.3,114673097,114673272,1383.331429,m551,175,-,16,"153,4",1.002614,1379.724306
4,AMPD1,Groupe 3,chr1,114673097,114695546,16,NM_000036.3,114673097,114673272,1573.497143,m552,175,-,16,"181,9",1.188889,1323.502270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231723,MYO18B,Groupe 3,chr22,25742187,26031045,44,NM_032608.7,26030442,26031045,129.529801,t551,603,+,44,"164,5",1.230366,105.277413
231724,MYO18B,Groupe 3,chr22,25742187,26031045,44,NM_032608.7,26030442,26031045,143.490066,t553,603,+,44,"179,5",1.342558,106.878116
231725,MYO18B,Groupe 3,chr22,25742187,26031045,44,NM_032608.7,26030442,26031045,126.600993,t554,603,+,44,"165,7",1.239342,102.151797
231726,MYO18B,Groupe 3,chr22,25742187,26031045,44,NM_032608.7,26030442,26031045,103.644040,t555,603,+,44,"141,4",1.057592,98.000057


In [32]:
profondeur_grouped.sort_values(by=['Groupe', 'symbol'], inplace=True)

In [33]:
profondeur_grouped

,symbol,Groupe,chrom,chromStart,chromEnd,exonCount,NM,exonStarts,exonEnds,profondeur_moy,échantillon,len_exon,strand,exon,prof_total,ratio,profondeur_norm
136320,ABHD5,Groupe 1,chr3,43690937,43722725,7,NM_016006.6,43690937,43691039,8.039216,m548,102,+,1,153,1.000000,8.039216
136321,ABHD5,Groupe 1,chr3,43690937,43722725,7,NM_016006.6,43690937,43691039,7.892157,m549,102,+,1,169,1.104575,7.144970
136322,ABHD5,Groupe 1,chr3,43690937,43722725,7,NM_016006.6,43690937,43691039,4.186275,m550,102,+,1,"159,9",1.045098,4.005629
136323,ABHD5,Groupe 1,chr3,43690937,43722725,7,NM_016006.6,43690937,43691039,9.098039,m551,102,+,1,"153,4",1.002614,9.074316
136324,ABHD5,Groupe 1,chr3,43690937,43722725,7,NM_016006.6,43690937,43691039,9.019608,m552,102,+,1,"181,9",1.188889,7.586586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198315,RXYLT1,NaN,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,9.857143,t551,888,+,6,"164,5",1.230366,8.011550
198316,RXYLT1,NaN,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,10.624297,t553,888,+,6,"179,5",1.342558,7.913474
198317,RXYLT1,NaN,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,17.030371,t554,888,+,6,"165,7",1.239342,13.741464
198318,RXYLT1,NaN,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,12.667042,t555,888,+,6,"141,4",1.057592,11.977252


In [34]:
profondeur_grouped.to_csv('profondeur_grouped', sep='\t', index=False)

In [42]:
NaN = profondeur_grouped.loc[(profondeur_grouped['Groupe'].isna())]

In [43]:
NaN

,symbol,Groupe,chrom,chromStart,chromEnd,exonCount,NM,exonStarts,exonEnds,profondeur_moy,échantillon,len_exon,strand,exon,prof_total,ratio,profondeur_norm
202752,ADSS1,NaN,chr14,104724228,104747310,13,NM_152328.5,104724228,104724462,680.692308,m548,234,+,1,153,1.000000,680.692308
202753,ADSS1,NaN,chr14,104724228,104747310,13,NM_152328.5,104724228,104724462,162.662393,m549,234,+,1,169,1.104575,147.262403
202754,ADSS1,NaN,chr14,104724228,104747310,13,NM_152328.5,104724228,104724462,274.115385,m550,234,+,1,"159,9",1.045098,262.286766
202755,ADSS1,NaN,chr14,104724228,104747310,13,NM_152328.5,104724228,104724462,616.683761,m551,234,+,1,"153,4",1.002614,615.075720
202756,ADSS1,NaN,chr14,104724228,104747310,13,NM_152328.5,104724228,104724462,600.213675,m552,234,+,1,"181,9",1.188889,504.852624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198315,RXYLT1,NaN,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,9.857143,t551,888,+,6,"164,5",1.230366,8.011550
198316,RXYLT1,NaN,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,10.624297,t553,888,+,6,"179,5",1.342558,7.913474
198317,RXYLT1,NaN,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,17.030371,t554,888,+,6,"165,7",1.239342,13.741464
198318,RXYLT1,NaN,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,12.667042,t555,888,+,6,"141,4",1.057592,11.977252


In [45]:
NaN['symbol'].unique()

array(['ADSS1', 'CRPPA', 'MYL11', 'MYMK', 'MYMX', 'PGAM2', 'RXYLT1'],
      dtype=object)

In [46]:
symbol = pd.DataFrame(NaN['symbol'].unique())

In [47]:
symbol

,0
0,ADSS1
1,CRPPA
2,MYL11
3,MYMK
4,MYMX
5,PGAM2
6,RXYLT1


In [48]:
symbol.to_csv('symbol', sep='\t', index=False)

In [49]:
profondeur_grouped['Groupe'] = profondeur_grouped['Groupe'].fillna('Groupe 4')

In [50]:
profondeur_grouped.to_csv('profondeur_grouped', sep='\t', index=False)

## Normalisation

In [2]:
profondeur_exons_strand = pd.read_csv('profondeur_exons_strand', sep="\t")

In [3]:
profondeur_exons_strand

,chrom,chromStart,chromEnd,exonCount,symbol,NM,exonStarts,exonEnds,profondeur_moy,échantillon,len_exon,strand,exon,profondeur_moy_standard,profondeur_moy_normal
0,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,825.828571,m548,175,-,16,-0.189945,0.104767
1,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,648.268571,m549,175,-,16,-0.269016,0.082241
2,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,631.280000,m550,175,-,16,-0.276581,0.080086
3,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,1383.331429,m551,175,-,16,0.058322,0.175494
4,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,1573.497143,m552,175,-,16,0.143007,0.199619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113371,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,129.529801,t551,603,+,44,-0.500021,0.016433
113372,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,143.490066,t553,603,+,44,-0.493804,0.018204
113373,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,126.600993,t554,603,+,44,-0.501325,0.016061
113374,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,103.644040,t555,603,+,44,-0.511548,0.013149


In [4]:
profondeur_exons_strand = profondeur_exons_strand.drop(columns=['profondeur_moy_standard','profondeur_moy_normal'])

In [10]:
ratio = pd.read_csv('ratio.csv', sep=";")

In [11]:
ratio

,échantillon,prof_total,ratio
0,m548,153,1
1,m549,169,"1,104575163"
2,m550,"159,9","1,045098039"
3,m551,"153,4","1,002614379"
4,m552,"181,9","1,188888889"
5,m553,216,"1,411764706"
6,m554,"183,9","1,201960784"
7,m555,"192,5","1,258169935"
8,t548,"133,7",1
9,t549,"142,8","1,068062827"


In [12]:
profondeur_exons_norm = pd.merge( profondeur_exons_strand, ratio, on=['échantillon'], how='inner')

In [13]:
profondeur_exons_norm

,chrom,chromStart,chromEnd,exonCount,symbol,NM,exonStarts,exonEnds,profondeur_moy,échantillon,len_exon,strand,exon,prof_total,ratio
0,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,825.828571,m548,175,-,16,153,1
1,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,648.268571,m549,175,-,16,169,"1,104575163"
2,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,631.280000,m550,175,-,16,"159,9","1,045098039"
3,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,1383.331429,m551,175,-,16,"153,4","1,002614379"
4,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,1573.497143,m552,175,-,16,"181,9","1,188888889"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113371,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,129.529801,t551,603,+,44,"164,5","1,230366492"
113372,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,143.490066,t553,603,+,44,"179,5","1,342557966"
113373,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,126.600993,t554,603,+,44,"165,7","1,23934181"
113374,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,103.644040,t555,603,+,44,"141,4","1,057591623"


In [16]:
profondeur_exons_norm['ratio'] = profondeur_exons_norm['ratio'].str.replace(',', '.')
profondeur_exons_norm['ratio'] = profondeur_exons_norm['ratio'].astype(float)
profondeur_exons_norm['profondeur_norm'] = profondeur_exons_norm['profondeur_moy'].div(profondeur_exons_norm['ratio'])

In [17]:
profondeur_exons_norm

,chrom,chromStart,chromEnd,exonCount,symbol,NM,exonStarts,exonEnds,profondeur_moy,échantillon,len_exon,strand,exon,prof_total,ratio,profondeur_norm
0,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,825.828571,m548,175,-,16,153,1.000000,825.828571
1,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,648.268571,m549,175,-,16,169,1.104575,586.894032
2,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,631.280000,m550,175,-,16,"159,9",1.045098,604.039025
3,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,1383.331429,m551,175,-,16,"153,4",1.002614,1379.724306
4,chr1,114673097,114695546,16,AMPD1,NM_000036.3,114673097,114673272,1573.497143,m552,175,-,16,"181,9",1.188889,1323.502270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113371,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,129.529801,t551,603,+,44,"164,5",1.230366,105.277413
113372,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,143.490066,t553,603,+,44,"179,5",1.342558,106.878116
113373,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,126.600993,t554,603,+,44,"165,7",1.239342,102.151797
113374,chr22,25742187,26031045,44,MYO18B,NM_032608.7,26030442,26031045,103.644040,t555,603,+,44,"141,4",1.057592,98.000057


In [18]:
profondeur_exons_norm.to_csv('profondeur_exons_norm', sep='\t', index=False)

## Création du score 

In [2]:
profondeur_grouped = pd.read_csv('profondeur_grouped', sep="\t")

In [3]:
profondeur_grouped

,symbol,Groupe,chrom,chromStart,chromEnd,exonCount,NM,exonStarts,exonEnds,profondeur_moy,échantillon,len_exon,strand,exon,prof_total,ratio,profondeur_norm
0,ABHD5,Groupe 1,chr3,43690937,43722725,7,NM_016006.6,43690937,43691039,8.039216,m548,102,+,1,153,1.000000,8.039216
1,ABHD5,Groupe 1,chr3,43690937,43722725,7,NM_016006.6,43690937,43691039,7.892157,m549,102,+,1,169,1.104575,7.144970
2,ABHD5,Groupe 1,chr3,43690937,43722725,7,NM_016006.6,43690937,43691039,4.186275,m550,102,+,1,"159,9",1.045098,4.005629
3,ABHD5,Groupe 1,chr3,43690937,43722725,7,NM_016006.6,43690937,43691039,9.098039,m551,102,+,1,"153,4",1.002614,9.074316
4,ABHD5,Groupe 1,chr3,43690937,43722725,7,NM_016006.6,43690937,43691039,9.019608,m552,102,+,1,"181,9",1.188889,7.586586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231723,RXYLT1,Groupe 4,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,9.857143,t551,888,+,6,"164,5",1.230366,8.011550
231724,RXYLT1,Groupe 4,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,10.624297,t553,888,+,6,"179,5",1.342558,7.913474
231725,RXYLT1,Groupe 4,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,17.030371,t554,888,+,6,"165,7",1.239342,13.741464
231726,RXYLT1,Groupe 4,chr12,63779908,63809562,6,NM_014254.3,63808674,63809562,12.667042,t555,888,+,6,"141,4",1.057592,11.977252


In [6]:
from sklearn.preprocessing import StandardScaler

profondeur_grouped['seq_type'] = profondeur_grouped['échantillon'].apply(lambda x: 'polyA' if x.startswith('m') else 'total')

scaler = StandardScaler()
profondeur_grouped['std'] = scaler.fit_transform(profondeur_grouped['profondeur_norm'].values.reshape(-1, 1))

exon_scores = profondeur_grouped.groupby(['NM', 'exon', 'seq_type'])['std'].median().unstack()
exon_scores['score_delta'] = exon_scores['polyA'] - exon_scores['total']
exon_scores.reset_index(inplace=True)

gene_scores = profondeur_grouped.groupby(['NM', 'exon']).first().reset_index()
gene_scores = gene_scores[['NM', 'symbol', 'chrom', 'chromStart', 'chromEnd', 'exon', 'exonCount', 'Groupe']]
gene_scores = gene_scores.merge(exon_scores[['NM', 'exon', 'score_delta']], on=['NM', 'exon'])

gene_scores['best'] = ['polyA' if diff > 0 else ('total' if diff < 0 else 'egal') for diff in gene_scores['score_delta']]
gene_scores_sorted = gene_scores.sort_values(by=['Groupe', 'symbol'])

gene_scores_sorted


,NM,symbol,chrom,chromStart,chromEnd,exon,exonCount,Groupe,score_delta,best
4402,NM_016006.6,ABHD5,chr3,43690937,43722725,1,7,Groupe 1,-0.001064,total
4403,NM_016006.6,ABHD5,chr3,43690937,43722725,2,7,Groupe 1,-0.002783,total
4404,NM_016006.6,ABHD5,chr3,43690937,43722725,3,7,Groupe 1,-0.003496,total
4405,NM_016006.6,ABHD5,chr3,43690937,43722725,4,7,Groupe 1,-0.004773,total
4406,NM_016006.6,ABHD5,chr3,43690937,43722725,5,7,Groupe 1,-0.001500,total
...,...,...,...,...,...,...,...,...,...,...
4223,NM_014254.3,RXYLT1,chr12,63779908,63809562,2,6,Groupe 4,0.007582,polyA
4224,NM_014254.3,RXYLT1,chr12,63779908,63809562,3,6,Groupe 4,0.012750,polyA
4225,NM_014254.3,RXYLT1,chr12,63779908,63809562,4,6,Groupe 4,0.009542,polyA
4226,NM_014254.3,RXYLT1,chr12,63779908,63809562,5,6,Groupe 4,0.012861,polyA


In [7]:
counts = gene_scores['best'].value_counts()

counts

best
total    3474
polyA    1893
egal      109
Name: count, dtype: int64

In [8]:
gene_scores.to_csv('gene_scores', sep='\t', index=False)